In [2]:
import copy

import numpy as np
import pandas as pd

from sklearn.model_selection import KFold
from skopt.space import Real, Categorical, Integer

import sys
sys.path.append('G:\내 드라이브\대학\대외\2023\연구원\연구과제\Ensemble-Kernel')

from Ensemble_Kernel_New import split_data,prepare_response_variable, c_o, nom, new_kernel, train_predict_fastKernelSurvivalSVM_bayesian, train_predict_fastKernelSurvivalSVM_clinical_bayesian, one_hot_encode_columns, remove_high_vif_columns, convert_categorical_to_boolean, new_kernel_no_abs, compare_kernels_with_abs_and_no_abs, train_predict_fastKernelSurvivalSVM_bayesian_no_abs, train_predict_fastKernelSurvivalSVM, train_predict_fastKernelSurvivalSVM_AFT, new_kernel_AFT, train_predict_fastKernelSurvivalSVM_bayesian_AFT, train_predict_fastKernelSurvivalSVM_clinical, train_predict_fastKernelSurvivalSVM_clinical_bayesian, C_index_fastKernelSurvivalSVM
from sksurv.kernels import clinical_kernel

from sksurv.metrics import concordance_index_censored
from sksurv.svm import FastSurvivalSVM, FastKernelSurvivalSVM
from sklearn.model_selection import ShuffleSplit, GridSearchCV

### 전처리

In [3]:
data = pd.read_csv("veteran_data.csv")

df = data.drop(['Unnamed: 0'], axis=1)
df = df.astype({'trt' : 'category', 'celltype' : 'category', 'prior' : 'category'})
df.replace({'celltype' : {'squamous' : 0, 'large' : 1, 'smallcell' : 2, 'adeno' : 3}}, inplace = True)

df = df.rename(columns={'age':'Age',
                        'time':'OS',
                        'status':'Status'})

#onehot encode
df_onehot, categorical_columns = one_hot_encode_columns(df)

columns_to_convert=['celltype_1', 'celltype_2','celltype_0', 'prior_10', 'trt_2']

df_onehot[columns_to_convert] = df_onehot[columns_to_convert].astype('category')

df_onehot

,OS,Status,karno,diagtime,Age,celltype_1,celltype_2,celltype_0,prior_10,trt_2
0,72,1,60,7,69,0,0,1,0,0
1,411,1,70,5,64,0,0,1,1,0
2,228,1,60,3,38,0,0,1,0,0
3,126,1,60,9,63,0,0,1,1,0
4,118,1,70,11,65,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...
132,133,1,75,1,65,1,0,0,0,1
133,111,1,60,5,64,1,0,0,0,1
134,231,1,70,18,67,1,0,0,1,1
135,378,1,80,4,65,1,0,0,0,1


In [4]:
#train, test split
x_train, x_test, target_train, target_test = split_data(df_onehot)

y_train = prepare_response_variable(target_train)
y_test = prepare_response_variable(target_test)

#Linear
x_train_linear=x_train.drop(['OS','Status'],axis=1)
x_test_linear=x_test.drop(['OS','Status'],axis=1)

In [5]:
x_train

,karno,diagtime,Age,celltype_1,celltype_2,celltype_0,prior_10,trt_2,OS,Status
90,70,22,36,0,1,0,1,1,103,0
109,99,3,57,0,0,0,0,1,83,0
21,60,5,67,0,1,0,0,0,97,0
71,80,3,48,0,0,1,0,1,87,0
20,40,3,55,0,1,0,0,0,123,0
...,...,...,...,...,...,...,...,...,...,...
51,80,5,64,0,0,0,0,0,162,1
86,60,13,70,0,0,1,1,1,44,1
74,70,7,50,0,0,1,1,1,991,1
48,80,2,38,0,0,0,0,0,117,1


In [6]:
#KFold setting
cv=KFold(n_splits = 5, shuffle=True, random_state=36)

param_space = {
    'alpha': Real(1e-6, 1e+6, 'log-uniform'),
}
param_grid = {'alpha': 2. ** np.arange(-12, 13, 2)}

### Cross Validation

In [7]:
#랜덤하게 순서 섞기
x_train = x_train.sample(frac=1).reset_index(drop=True)

# train data를 그룹으로 나누기
num_groups = 3
group_size = len(x_train) // num_groups

x_groups = []

for i in range(num_groups):
    if i < num_groups - 1:
        group = x_train.iloc[i * group_size:(i + 1) * group_size]
    else:
        group = x_train.iloc[i * group_size:]
    
    x_groups.append(group)

In [11]:
cindex=[]

for i in range(len(x_groups)):
    temp = []
    for j in range(len(x_groups)):
        if i != j:
            temp.append(x_groups[j])
    #train = train data(등분한 것 중 1개를 제외한 나머지)
    train=pd.concat(temp)

    #validation = validation data(등분한 것 중 1개)
    validation = x_groups[i]

    train_target=train[['Status','OS']]
    test_target=validation[['Status','OS']]

    train_y=prepare_response_variable(train_target)
    test_y=prepare_response_variable(test_target)
    #data정의 -> train, validation, train_y, test_y

    result = C_index_fastKernelSurvivalSVM(train, train_y, validation, test_y, param_grid, keywords = ['Age', 'Sex'], drop=False, coef_drop=None)
    
    cindex.append(result[0])
    print(result[0]) #생략 가능

0.8001132502831257
0.8244587478057344
0.7734608487746563


[0.8001132502831257, 0.8244587478057344, 0.7734608487746563]

In [13]:
# cindex 평균 구하기
np.mean(cindex)

0.7993442822878388